# **Előkészületek**

In [1]:
!pip install -q torch transformers transformers langchain sentence-transformers tqdm openpyxl openai pandas datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.8/812.8 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.9/266.9 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
from tqdm.auto import tqdm
import pandas as pd
from typing import Optional, List, Tuple
import json
import datasets

#megelőzzük a Pandasban lévő hosszú szövegek levágását
pd.set_option("display.max_colwidth", None)

In [4]:
from huggingface_hub import notebook_login

#bejelentkezünk a HF HUB API tokennel
notebook_login()

# **Adathalmaz betöltése**

### Wikipédiás fájlok betöltése

In [7]:
ds= datasets.load_dataset("wikipedia", "20220301.simple", split="train")

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1461: FutureWarning: The repository for wikipedia contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/wikipedia
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [ ]:
ds

In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document as LangchainDocument

langchain_docs = [LangchainDocument(page_content=doc["text"], metadata={"source": doc["title"]}) for doc in tqdm(ds)]
# behelyezi a szöveget és a forrásokat a langchainDocument formátumú listába, itt átneveztem az oszlopot title-re

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=200,
    add_start_index=True,
    separators=["\n\n", "\n", ".", " ", ""],
)
#egy szegmentáló algoritmus mely a megadott paraméterek alapján vágja bizonyos részekre a szöveget
#majd ezeket akár külön szeparátorokkal kezeli és különböző részekre bontja


docs_processed = []
for doc in langchain_docs:
    docs_processed += text_splitter.split_documents([doc])

#miután bekalibráltuk a szegmentáló algoritmust, azután végighaladunk az általunk létrehozott dokumentum listán

  0%|          | 0/205328 [00:00<?, ?it/s]

### Alternatív betöltés

In [ ]:
ds = datasets.load_dataset("m-ric/huggingface_doc", split="train")
#betöltjük az adott adathalmazt

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document as LangchainDocument

langchain_docs = [LangchainDocument(page_content=doc["text"], metadata={"source": doc["source"]}) for doc in tqdm(ds)]
# behelyezi a szöveget és a forrásokat a langchainDocument formátumú listába

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=200,
    add_start_index=True,
    separators=["\n\n", "\n", ".", " ", ""],
)
#egy szegmentáló algoritmus mely a megadott paraméterek alapján vágja bizonyos részekre a szöveget
#majd ezeket akár külön szeparátorokkal kezeli és különböző részekre bontja


docs_processed = []
for doc in langchain_docs:
    docs_processed += text_splitter.split_documents([doc])

#miután bekalibráltuk a szegmentáló algoritmust, azután végighaladunk az általunk létrehozott dokumentum listán

  0%|          | 0/2647 [00:00<?, ?it/s]

## Tesztelő LLM Kliens hívás

In [9]:
from huggingface_hub import InferenceClient


repo_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"

llm_client = InferenceClient(
    model=repo_id,
    timeout=120,
)
#meghívjuk az InferenceClient modult a huggingface könyvtárból, majd beállítjuk paraméterenek a mistarlAI modelljét
#továbbá korlátozzuk a kérés maximális időtúllépését másodpercekben

def call_llm(inference_client: InferenceClient, prompt: str):
    response = inference_client.post(
        json={
            "inputs": prompt,
            "parameters": {"max_new_tokens": 1000},
            "task": "text-generation",
        },
    )
    return json.loads(response.decode())[0]["generated_text"]

#létrehozunk egy metódust, melyben meghívhatjuk a definiált LLM klienst

call_llm(llm_client, "This is a test context")

'This is a test context for the `@mui/material` library.\n\n## Installation\n\n```sh\nnpm install @mui/material\n```\n\n## Usage\n\n```jsx\nimport React from \'react\';\nimport { Button } from \'@mui/material\';\n\nfunction App() {\n  return (\n    <div className="App">\n      <Button variant="contained" color="primary">\n        Hello World\n      </Button>\n    </div>\n  );\n}\n\nexport default App;\n```\n\n## Documentation\n\n- [Material-UI](https://material-ui.com/)\n- [Material Design](https://material.io/)'

In [10]:
#Tesztelésként nézzük meg mit reagál egy kérdésre és hogyan tud választ adni rá
result =call_llm(llm_client, "What is the meaning of life?")
print(result)

What is the meaning of life?

This is a question that has puzzled philosophers, theologians, and scientists for centuries. It is a question that has been asked in many different ways, and it has been given many different answers.

Some people believe that the meaning of life is to seek happiness and fulfillment. Others believe that it is to serve a higher power or to follow a set of moral principles. Still others believe that the meaning of life is to learn and grow, to experience new things and to become the best version of ourselves that we can be.

Ultimately, the meaning of life is a personal and subjective question. It is up to each individual to decide what gives their life meaning and purpose. For some, this may be a religious or spiritual belief. For others, it may be a career or a hobby. And for others still, it may be a relationship or a family.

No matter what gives your life meaning, it is important to remember that the journey is just as important as the destination. The e

In [11]:
#Most pedig nézzük meg mit mond a Hell-el kapcsolatban
result =call_llm(llm_client, "Can you tell me about Hell Energy?")
print(result)

Can you tell me about Hell Energy?

Hell Energy is a Hungarian energy drink brand that was founded in 2006. The company produces a range of energy drinks, including its flagship product, Hell Energy Original, which is available in over 50 countries worldwide.

Hell Energy Original is a carbonated energy drink that contains caffeine, taurine, vitamins, and other ingredients that are designed to provide a quick energy boost. The drink is known for its distinctive black and red packaging and its bold, edgy marketing campaigns.

In addition to its original energy drink, Hell Energy also produces a range of other energy drinks, including sugar-free and zero-calorie options, as well as flavored varieties such as apple, cherry, and mango. The company also produces a line of energy shots and energy gum.

Hell Energy has gained a reputation for its high-quality products and its commitment to innovation. The company has won numerous awards for its marketing campaigns and has been recognized as o